In [4]:
import warnings
with warnings.catch_warnings():
    ## ignore future warnings from too-advanced numpy version
    warnings.simplefilter("ignore")    
    from keras.models import Sequential
    from keras.layers import Dense, Activation
    
from importlib import reload    

In [7]:
import energy
_ = reload(energy)

In [8]:
MAX_ROWS = 100000
weather = energy.Data.Weather(MAX_ROWS)
meter = energy.Data.Meter(MAX_ROWS)
buildings = energy.Data.Buildings()

In [11]:
combined_df = energy.merge_data(
    weather_df=weather.train, 
    meter_df=meter.train,
    buildings_df=buildings.data)

In [56]:
combined_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


In [44]:
rmsle = energy.define_rmsle(X_df.shape[0])

In [49]:
X_df = combined_df[['square_feet', 'meter', 'year_built', 'air_temperature']]
y = combined_df[['meter_reading']].values

nan_rowinds = X_df.apply(lambda row: energy.pd.isnull(row).any(), axis=1)
X_df = X_df[~nan_rowinds]
y = y[~nan_rowinds]
X = X_df.values

In [51]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=X.shape[1]))
model.add(Dense(1, activation='relu'))

In [52]:
model.compile(optimizer='rmsprop',
              loss='mse', metrics=['mse'])

In [53]:
model.fit(X_df.values, y, epochs=10, batch_size=32)

Epoch 1/10
30977/30977 [==============================] - 2s 49us/step - loss: 247995.5607 - mse: 247995.4531
Epoch 2/10
30977/30977 [==============================] - 1s 44us/step - loss: 247995.5598 - mse: 247995.6094
Epoch 3/10
30977/30977 [==============================] - 1s 46us/step - loss: 247995.5595 - mse: 247995.6094
Epoch 4/10
30977/30977 [==============================] - 1s 45us/step - loss: 247995.5600 - mse: 247995.5312
Epoch 5/10
30977/30977 [==============================] - 1s 45us/step - loss: 247995.5596 - mse: 247995.5156
Epoch 6/10
30977/30977 [==============================] - 1s 46us/step - loss: 247995.5605 - mse: 247995.5469
Epoch 7/10
30977/30977 [==============================] - 1s 45us/step - loss: 247995.5591 - mse: 247995.5000
Epoch 8/10
30977/30977 [==============================] - 1s 46us/step - loss: 247995.5599 - mse: 247995.2031
Epoch 9/10
30977/30977 [==============================] - 1s 47us/step - loss: 247995.5595 - mse: 247995.7969
Epoch 10/1